In [1]:
import pickle
from collections import Counter
from datetime import time, datetime

from tqdm import tqdm 

import attr
import pandas as pd

from typing import Dict, List

from pprint import pprint

import wrestling
from wrestling.base import Mark

In [2]:
with open('all_matches.p', 'rb') as f:
    loaded_matches = pickle.load(f)
print(len(loaded_matches))

# use single match and delete for space saving then try on all later
match = loaded_matches[1000]

del loaded_matches

match

815758


{'id': '1567603412:2019-2020:0:1576851265.755613:1567603412.1838334878321:2019-12-20 14:16:56 +0000',
 'date': '2019-12-20',
 'weight': '132',
 'scoring_events': '1*11*0*T2*Green#2*0*0*R2*Red#2*30*0*P1*Red#2*50*0*N2*Red#5*17*0*R2*Green#5*17*0*P1*Green#5*48*0*P1*Red#6*9*0*E1*Red',
 'winner': 1,
 'result': 'Decision 7-5',
 'red_wrestler': '1567603412.1838334878321#Ivy, Tre#(null)#Ada#(null)',
 'green_wrestler': '(null)#Doe, John#(null)#Niceville#(null)#',
 'level': 'High School Boys'}

In [3]:
# request events
with open('all_events.p', 'rb') as f:
    events = pickle.load(f) 
print(len(events))

55215


In [5]:
# request roster
with open('all_rosters.p', 'rb') as f:
    rosters = pickle.load(f)
print(len(rosters))

69012


In [6]:
def parse_date(match_id):
    regex = r"(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})"
    return datetime.strptime(re.search(regex, match_id).group(1), "%Y-%m-%d %H:%M:%S")

In [7]:
def get_focus(match):
    red = match["red_wrestler"]
    green = match["green_wrestler"]
    if red.startswith(match['our_wrestler']):
        return "red"
    elif green.startswith(match['our_wrestler']):
        return "green"
    else:
        # this is a problem
        pass
    
def parse_name(wrestler_string) -> str:
    regex_search = r"#(.*?)#"
    # Not flipped to maintain format across platform
    name = re.findall(regex_search, wrestler_string)[0].split(",")
    return f"{name[0].strip()}, {name[1].strip()}" if len(name) > 1 else None


def parse_team(wrestler_string) -> str:
    regex_search = r"#(.*?)#"
    team = re.findall(regex_search, wrestler_string)[1]
    if "(" in team:
        return team.split("(")[0]
    return team

In [11]:
def create_focus_wrestler(wrestler_str: str, wrestler_list: List[Dict]):
    wrestler_name = parse_name(wrestler_str)
    # find wrestler in wrestler_list
    # no default because wrestler MUST be found
    raw_wrestler = next(w for w in wrestler_list if w["name"] == wrestler_name)
    # create wrestler
    wrestler = wrestling.Wrestler(
        name=wrestler_name,
        team=parse_team(wrestler_str),
        grade=wrestling.Mark(wrestler['grade'])
    )
    return wrestler


def create_opponent_wrestler(wrestler_str: str):
    # create wrestler
    wrestler = wrestling.Wrestler(
        name=parse_name(wrestler_str),
        team=parse_team(wrestler_str),
    )
    return wrestler

In [8]:
def parse_api_scoring_events(event_string):
    main_list = []
    for action in event_string.split('#'):
        if len(action.split('*')) == 1 or action == '****':
            continue
        A = action.split('*')
        if not A[0].isdigit() or int(A[0]) not in range(0, 60):
            minute = 0
        else:
            minute = int(A[0])
        if not A[1].isdigit() or int(A[1]) not in range(0, 60):
            second = 0
        else:
            second = int(A[1])
        period = int(A[2]) + 1 if A[2] else 0
        label = A[3]
        color = A[4].lower() if A[4] in {'Red', 'Green'} else 'green'
        main_list.append((minute, second, period, label, color))
    return tuple(main_list)

In [9]:
def create_hs_scoring(event_string):
    ts_gen = parse_api_scoring_events(event_string)
    temp = []
    for action in ts_gen:
        sc = wrestling.HSScoring(
            initiator=action[4],
            focus_color=focus,
            period=action[2],
            time_stamp=time(minute=action[0], second=action[1]),
            label=wrestling.HSLabel(action[3]),
        )
        temp.append(sc)
    return tuple(sorted(temp))
    
    
def create_college_scoring(event_string):
    ts_gen = parse_api_scoring_events(event_string)
    temp = []
    for action in ts_gen:
        sc = wrestling.CollegeScoring(
            initiator=action[4],
            focus_color=focus,
            period=action[2],
            time_stamp=time(minute=action[0], second=action[1]),
            label=wrestling.CollegeLabel(action[3]),
        )
        temp.append(sc)
    return tuple(sorted(temp))

In [10]:
def create_event(event_id, event_list):
    # find event_id in event_list
    raw_event = next((e for e in event_list if e["id"] == event_id), None)
    # if event exists create it, otherwise create default
    if raw_event:
        event = wrestling.Event(
            name=raw_event['name'],
            kind=wrestling.Mark(raw_event['kind']),
        )
        return event
    else:
        default = wrestling.Event(
            name='Default Event',  # or leave empty and will be set to Generic Event by converter
            kind=wrestling.Mark('Tournament'),
        )
        return default

In [13]:
def parse_result(match):
    method = match['result']
    if method.startswith('Tech'):
        second = 'T'
    elif method.startswith('Fall'):
        second = 'F'
    elif method.startswith('Maj'):
        second = 'M'
    elif method.startswith('Dec'):
        second = 'D'
    else:
        return 'NC'
        
    binary = match['winner']
    if binary == 0:
        first = 'L'
    elif binary == 1:
        first = 'W'
    else:
        raise ValueError(f"unexpected winner{match['winner']}")
    return f"{first}{second}"

In [12]:
def create_college_match(match: Dict, event_list: List[Dict], wrestler_list: List[Dict]):
    # call parsers and then creators
    # this is the final function
    # we only directly call this for each match :)
    F = get_focus(match)
    m = wrestling.CollegeMatch(
        id=match['id'],
        event=create_event(match['event_id'], event_list),
        date=parse_date(match['id']),
        result=wrestling.Result.LT,  # parse result function needed!
        focus=create_focus_wrestler(match['green_wrestler'] if F == 'green' else match['red_wrestler']),
        opponent=create_opponent_wrestler(match['green_wrestler'] if F == 'red' else match['red_wrestler']),
        weight=wrestling.Mark(match['weight']),
        time_series=create_college_scoring(match['scoring_events']),
    )
    return m


def create_hs_match(match_dict):
    # call parsers and then creators
    # this is the final function
    # we only directly call this for each match :)
    F = get_focus(match)
    m = wrestling.HSMatch(
        id=match['id'],
        event=create_event(match['event_id'], event_list),
        date=parse_date(match['id']),
        result=wrestling.Result.LT,  # parse result function needed!
        focus=create_focus_wrestler(match['green_wrestler'] if F == 'green' else match['red_wrestler']),
        opponent=create_opponent_wrestler(match['green_wrestler'] if F == 'red' else match['red_wrestler']),
        weight=wrestling.Mark(match['weight']),
        time_series=create_hs_scoring(match['scoring_events']),
    )
    return m